In [1]:
import sys
import pandas as pd
import numpy as np
from IPython.display import display
# do not truncate the dataframe column width
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None

sys.path.append('F:/repo/gpsbeam')

from src.config.data_config import DataConfig
from src.config.cnn_ed_rnn_model_config import ModelConfig
from src.config.experiment_config import ExperimentConfig
from src.experiment import RunExperiment

def combined_metric(row):
    loss_db = np.mean(row['pred0_top1_mean_power_loss_db'] + row['pred1_top1_mean_power_loss_db']\
             + row['pred2_top1_mean_power_loss_db'] + row['pred3_top1_mean_power_loss_db'])
    accuracy = np.mean(row['pred0_top1_test_acc_percent'] + row['pred1_top1_test_acc_percent']\
                      + row['pred2_top1_test_acc_percent'] + row['pred3_top1_test_acc_percent']) / 100
    invert_loss_db = 1/loss_db
    return (invert_loss_db * 0.5) + (accuracy * 0.5)

def df_apply_combined_metric(df):
    key = 'combined_loss_db_and_acc'
    df[key] = df.apply(combined_metric, axis=1)
    df = df.sort_values(by=key, ascending=False)
    return df


# Sequence Length Variation

In [2]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=["unit2to1_vector", "unit2_height_log"],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_seq_len',
                  exp_dict={
                       "seq_len": [3, 4, 5, 6, 7, 8, 9],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:11:28.733 | DEBUG    | src.experiment:run:40 - updated model_config | seq_len to 3
2025-05-20 15:11:28.733 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:11:28.734 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:11:28.735 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2to1_vector', 'unit2_height_log']
2025-05-20 15:11:51.689 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_adjusted.hkl
2025-05-20 15:11:51.689 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                 

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_11_51\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_11_51\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:15:26.772 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:15:26.788 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_11_51\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_11_51\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 81.78it/s]
2025-05-20 15:15:26.878 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.738 0.771 0.76  0.725]
 [0.956 0.967 0.959 0.945]
 [0.992 0.994 0.994 0.986]]
2025-05-20 15:15:31.560 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_11_51\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:15:31.746 

...

test: 100%|██████████| 3/3 [00:00<00:00, 64.87it/s]
2025-05-20 15:26:09.464 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.753 0.767 0.75  0.715]
 [0.962 0.972 0.962 0.952]
 [0.985 0.99  0.987 0.977]]
2025-05-20 15:26:14.131 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_23_59\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:26:14.263 

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_26_14\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_26_14\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:28:27.717 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:28:27.734 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_26_14\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_26_14\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 2/2 [00:00<00:00, 39.94it/s]
2025-05-20 15:28:27.817 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.743 0.754 0.733 0.704]
 [0.953 0.966 0.952 0.933]
 [0.987 0.992 0.991 0.98 ]]
2025-05-20 15:28:32.939 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\05-20-2025_15_26_14\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:28:33.106 

In [6]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_seq_len_20052025_151128\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,seq_len,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
0,3,32,"['unit2to1_vector', 'unit2_height_log']",23,2.017,0.99,0.249,0.226,0.242,0.483,1.785,1.773,1.771,1.768,94.422,94.460,94.466,94.474,1.957,1.946,1.946,1.947,93.886,93.917,93.919,93.916,2.171,2.151,2.150,2.148,93.216,93.278,93.281,93.287,2.665,2.667,2.683,2.683,91.673,91.665,91.615,91.615,4.242,4.370,4.357,4.370,86.743,86.344,86.383,86.344,94.983,97.057,97.057,94.983,98.939,99.518,99.035,98.312,99.759,99.662,99.469,99.132,73.8,95.6,99.2,77.1,96.7,99.4,76.0,95.9,99.4,72.5,94.5,98.6,1.913667
1,4,32,"['unit2to1_vector', 'unit2_height_log']",23,1.568,0.99,0.247,0.241,0.259,0.586,1.837,1.822,1.821,1.820,94.260,94.306,94.311,94.312,2.033,2.007,2.004,2.006,93.648,93.729,93.738,93.732,2.246,2.240,2.239,2.238,92.981,93.000,93.005,93.006,2.856,2.843,2.835,2.835,91.075,91.116,91.140,91.141,4.469,4.480,4.469,4.458,86.034,86.000,86.034,86.070,95.362,96.687,96.167,93.895,99.053,99.432,98.817,98.060,99.716,99.669,99.527,99.006,73.8,95.7,99.2,75.5,96.3,99.6,73.6,95.8,99.4,70.1,94.0,98.6,1.840094
2,5,32,"['unit2to1_vector', 'unit2_height_log']",23,1.649,0.99,0.278,0.251,0.285,0.640,1.856,1.848,1.851,1.858,94.199,94.226,94.214,94.195,2.017,2.004,2.005,2.001,93.698,93.737,93.734,93.746,2.255,2.245,2.240,2.239,92.954,92.985,93.000,93.002,2.813,2.816,2.813,2.816,91.208,91.199,91.208,91.199,4.471,4.521,4.528,4.532,86.029,85.871,85.851,85.837,94.969,96.298,95.8

# Feature Variation


In [3]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=["unit2to1_vector", "unit2_height_log"],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_feature_input',
                  exp_dict={
                       "model_input_column_list": [
                           ['unit2_loc_minmax_norm'], 
                           ['unit2_height_log'],
                           ['unit2_height'],      
                           ["unit2to1_vector", "unit2_height"],                 
                           ["unit2to1_vector", "unit2_height_log"],
                        #    ['unit1_beam_idx_32']
                           ],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:28:33.590 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2_loc_minmax_norm']
2025-05-20 15:28:33.590 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:28:33.590 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:28:55.330 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_adjusted.hkl
2025-05-20 15:28:55.330 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                                : adjusted
                    Portion Percentage                              : 100
           

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_28_55\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_28_55\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:30:58.051 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:30:58.051 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_28_55\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_28_55\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 90.95it/s]
2025-05-20 15:30:58.124 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.648 0.637 0.634 0.625]
 [0.898 0.903 0.897 0.882]
 [0.967 0.971 0.969 0.966]]
2025-05-20 15:31:02.772 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_28_55\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:31:0

...

2025-05-20 15:37:18.413 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:37:18.414 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_35_06\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth
test: 100%|██████████| 3/3 [00:00<00:00, 89.83it/s]
2025-05-20 15:37:18.481 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.638 0.647 0.625 0.608]
 [0.913 0.932 0.925 0.914]
 [0.972 0.981 0.977 0.969]]
2025-05-20 15:37:23.119 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_35_06\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Lo

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_37_23\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_37_23\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:39:23.119 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:39:23.122 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_37_23\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_37_23\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 82.26it/s]
2025-05-20 15:39:23.192 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.753 0.767 0.75  0.715]
 [0.962 0.972 0.962 0.952]
 [0.985 0.99  0.987 0.977]]
2025-05-20 15:39:28.210 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\05-20-2025_15_37_23\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:39:2

In [7]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_feature_input_20052025_152833\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,model_input_column_list,num_classes,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
4,"['unit2to1_vector', 'unit2_height_log']",32,23,1.579,0.99,0.328,0.345,0.397,0.836,1.907,1.897,1.900,1.904,94.040,94.073,94.061,94.049,2.084,2.086,2.095,2.104,93.488,93.482,93.454,93.425,2.393,2.394,2.394,2.396,92.523,92.518,92.520,92.512,2.996,3.007,3.029,3.050,90.637,90.604,90.535,90.468,4.668,4.706,4.774,4.821,85.413,85.293,85.082,84.934,95.268,96.000,95.805,93.415,98.878,99.171,98.244,97.317,99.610,99.463,99.317,98.634,75.3,96.2,98.5,76.7,97.2,99.0,75.0,96.2,98.7,71.5,95.2,97.7,1.754829
0,['unit2_loc_minmax_norm'],32,23,3.235,0.99,0.726,0.805,0.889,1.224,1.833,1.838,1.838,1.838,94.273,94.256,94.256,94.256,2.029,2.010,2.033,2.038,93.660,93.718,93.646,93.633,2.302,2.248,2.259,2.280,92.806,92.976,92.942,92.877,2.836,2.781,2.802,2.820,91.139,91.309,91.244,91.186,4.180,4.171,4.225,4.270,86.939,86.966,86.797,86.657,90.146,90.293,91.268,90.488,96.195,96.293,96.537,96.098,98.634,98.390,98.195,97.854,64.8,89.8,96.7,63.7,90.3,97.1,63.4,89.7,96.9,62.5,88.2,96.6,1.409212
3,"['unit2to1_vector', 'unit2_height']",32,23,1.929,0.99,0.776,0.729,0.777,1.109,2.753,2.708,2.706,2.711,91.396,91.537,91.544,91.527,3.187,3.118,3.100,3.101,90.040,90.258,90.311,90.308,3.742,3.734,3.720,3.716,88.306,88.332,88.373,88.389,4.621,4.559,4.548,4.549,85.558,85.753,85.787,85.784,6.614,6.519,6.507,6.516,79.331,79.630,79.666,79.639,86.780,88.683,88.439,86.634,95.415,95.707,95.659,94.

# Var Dataset Split Method

In [4]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=["unit2to1_vector", "unit2_height_log"],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_spliting_method',
                  exp_dict={
                       "splitting_method": [
                           "sequential",
                           "adjusted",
                           ],
                       "num_classes": [32]
                  })).run()

2025-05-20 15:39:28.744 | DEBUG    | src.experiment:run:36 - updated data_config |splitting_method to sequential
2025-05-20 15:39:28.744 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 32
2025-05-20 15:39:28.744 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 32
2025-05-20 15:39:28.744 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2to1_vector', 'unit2_height_log']
2025-05-20 15:39:50.304 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_sequential_shuffle_False_.hkl
2025-05-20 15:39:50.304 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_39_50\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_sequential.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_39_50\dl_generated/model_measurement/label_distribution_scenario_23_sampling_sequential.pdf


2025-05-20 15:41:51.036 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:41:51.053 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_39_50\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_39_50\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 88.97it/s]
2025-05-20 15:41:51.125 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.545 0.517 0.509 0.499]
 [0.942 0.946 0.941 0.934]
 [0.998 0.998 0.996 0.994]]
2025-05-20 15:41:55.763 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_39_50\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:41

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_42_17\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_42_17\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf
Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_42_17\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


2025-05-20 15:44:17.087 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 32
2025-05-20 15:44:17.099 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_42_17\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_32_.pth
test: 100%|██████████| 3/3 [00:00<00:00, 64.95it/s]
2025-05-20 15:44:17.168 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.753 0.767 0.75  0.715]
 [0.962 0.972 0.962 0.952]
 [0.985 0.99  0.987 0.977]]
2025-05-20 15:44:21.354 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\05-20-2025_15_42_17\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppDat

In [8]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_spliting_method_20052025_153928\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
df = df_apply_combined_metric(df)
df

,splitting_method,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent,combined_loss_db_and_acc
1,adjusted,32,"['unit2to1_vector', 'unit2_height_log']",23,1.579,0.99,0.328,0.345,0.397,0.836,1.907,1.897,1.900,1.904,94.040,94.073,94.061,94.049,2.084,2.086,2.095,2.104,93.488,93.482,93.454,93.425,2.393,2.394,2.394,2.396,92.523,92.518,92.520,92.512,2.996,3.007,3.029,3.050,90.637,90.604,90.535,90.468,4.668,4.706,4.774,4.821,85.413,85.293,85.082,84.934,95.268,96.000,95.805,93.415,98.878,99.171,98.244,97.317,99.610,99.463,99.317,98.634,75.3,96.2,98.5,76.7,97.2,99.0,75.0,96.2,98.7,71.5,95.2,97.7,1.754829
0,sequential,32,"['unit2to1_vector', 'unit2_height_log']",23,1.338,0.99,0.434,0.469,0.555,0.641,2.771,2.755,2.628,2.665,91.341,91.392,91.788,91.672,3.230,3.189,3.176,3.164,89.905,90.035,90.075,90.111,3.803,3.775,3.765,3.768,88.115,88.203,88.234,88.226,4.767,4.656,4.620,4.731,85.102,85.452,85.563,85.215,6.327,6.110,6.087,6.075,80.227,80.906,80.977,81.017,85.876,85.604,85.197,84.337,98.551,98.732,98.551,98.099,99.955,99.819,99.502,99.230,54.5,94.2,99.8,51.7,94.6,99.8,50.9,94.1,99.6,49.9,93.4,99.4,1.273209


# Var num_classes


In [5]:
run_obj = RunExperiment(data_config =DataConfig(scenario_num=23,
                                                splitting_method='adjusted',
                                                train_val_test_split_frac=[0.65,0.15, 0.2],
                                                seednum=42
                                                ), 
              model_config=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cuda',
                                       model_input_column_list=["unit2to1_vector", "unit2_height_log"],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       ),
              experiment_config=ExperimentConfig(
                  exp_folder_name='01_drone_cnn_ed_rnn_experiment_var_num_classes',
                  exp_dict={
                       "num_classes": [8, 16, 32, 64]
                  })).run()


2025-05-20 15:44:21.888 | DEBUG    | src.experiment:run:36 - updated data_config |num_classes to 8
2025-05-20 15:44:21.888 | DEBUG    | src.experiment:run:40 - updated model_config | num_classes to 8
2025-05-20 15:44:21.888 | DEBUG    | src.experiment:run:40 - updated model_config | model_input_column_list to ['unit2to1_vector', 'unit2_height_log']
2025-05-20 15:44:43.356 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam8_splitting_method_adjusted.hkl
2025-05-20 15:44:43.356 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                                : adjusted
                    Portion Percentage                              : 100


Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_44_43\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_44_43\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:46:36.989 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 8
2025-05-20 15:46:37.005 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_44_43\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_8_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_44_43\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 2/2 [00:00<00:00, 59.30it/s]
2025-05-20 15:46:37.074 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.868 0.872 0.856 0.84 ]
 [0.997 0.997 0.995 0.993]
 [1.    1.    0.999 0.998]]
2025-05-20 15:46:41.209 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_44_43\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:46:41.

...

test: 100%|██████████| 3/3 [00:00<00:00, 60.07it/s]
2025-05-20 15:51:07.543 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.753 0.767 0.75  0.715]
 [0.962 0.972 0.962 0.952]
 [0.985 0.99  0.987 0.977]]
2025-05-20 15:51:11.635 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_49_13\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:51:11.

Saving plot data to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_51_33\dl_generated/model_measurement/label_distribution_data_scenario_23_sampling_adjusted.csv
Saving plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_51_33\dl_generated/model_measurement/label_distribution_scenario_23_sampling_adjusted.pdf


2025-05-20 15:53:13.284 | DEBUG    | src.modelprep:_load_best_checkpoint:550 - Get Model: cnn-ed-gru-model
                         NUM CLASSES: 64
2025-05-20 15:53:13.284 | INFO     | src.modelprep:_load_best_checkpoint:553 - Model loaded from f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_51_33\dl_generated/model_checkpoint/arch_cnn-ed-gru-model_nclass_64_.pth


Saved plot to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_51_33\dl_generated/model_measurement/train_val_loss_vs_epochs.pdf


test: 100%|██████████| 3/3 [00:00<00:00, 56.47it/s]
2025-05-20 15:53:13.382 | DEBUG    | src.utils.metric_handler:calculate_final_metrics_batch_first:81 - 
[[0.571 0.61  0.601 0.571]
 [0.902 0.907 0.907 0.893]
 [0.96  0.964 0.965 0.958]]
2025-05-20 15:53:17.488 | INFO     | src.modelprep:_save_inference_detail:661 - Saved inference detail to f:/repo/gpsbeam\data/experiment_result/01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\05-20-2025_15_51_33\dl_generated/model_inference_result/test_epoch_00_inference_detail.csv
c:\Users\vendi\AppData\Local\pypoetry\Cache\virtualenvs\geobeam-3fwD9wo9-py3.9\lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
2025-05-20 15:53:17.

In [9]:
fname = r'F:\repo\gpsbeam\data\experiment_result\01_drone_cnn_ed_rnn_experiment_var_num_classes_20052025_154421\model_recap\exp_test_result_recap.csv'
df = pd.read_csv(fname)
# df = df_apply_combined_metric(df)
df

,num_classes,model_input_column_list,scenario_num,avg_loss,model_size_MiB,pred0_top1_mean_power_loss_db,pred1_top1_mean_power_loss_db,pred2_top1_mean_power_loss_db,pred3_top1_mean_power_loss_db,pred0_top1_n_beams_for_80pct_conf,pred1_top1_n_beams_for_80pct_conf,pred2_top1_n_beams_for_80pct_conf,pred3_top1_n_beams_for_80pct_conf,pred0_top1_overhead_saving_for_80pct_conf,pred1_top1_overhead_saving_for_80pct_conf,pred2_top1_overhead_saving_for_80pct_conf,pred3_top1_overhead_saving_for_80pct_conf,pred0_top1_n_beams_for_85pct_conf,pred1_top1_n_beams_for_85pct_conf,pred2_top1_n_beams_for_85pct_conf,pred3_top1_n_beams_for_85pct_conf,pred0_top1_overhead_saving_for_85pct_conf,pred1_top1_overhead_saving_for_85pct_conf,pred2_top1_overhead_saving_for_85pct_conf,pred3_top1_overhead_saving_for_85pct_conf,pred0_top1_n_beams_for_90pct_conf,pred1_top1_n_beams_for_90pct_conf,pred2_top1_n_beams_for_90pct_conf,pred3_top1_n_beams_for_90pct_conf,pred0_top1_overhead_saving_for_90pct_conf,pred1_top1_overhead_saving_for_90pct_conf,pred2_top1_overhead_saving_for_90pct_conf,pred3_top1_overhead_saving_for_90pct_conf,pred0_top1_n_beams_for_95pct_conf,pred1_top1_n_beams_for_95pct_conf,pred2_top1_n_beams_for_95pct_conf,pred3_top1_n_beams_for_95pct_conf,pred0_top1_overhead_saving_for_95pct_conf,pred1_top1_overhead_saving_for_95pct_conf,pred2_top1_overhead_saving_for_95pct_conf,pred3_top1_overhead_saving_for_95pct_conf,pred0_top1_n_beams_for_99pct_conf,pred1_top1_n_beams_for_99pct_conf,pred2_top1_n_beams_for_99pct_conf,pred3_top1_n_beams_for_99pct_conf,pred0_top1_overhead_saving_for_99pct_conf,pred1_top1_overhead_saving_for_99pct_conf,pred2_top1_overhead_saving_for_99pct_conf,pred3_top1_overhead_saving_for_99pct_conf,pred0_top1_reliability_power_loss_leq_1db_pct,pred1_top1_reliability_power_loss_leq_1db_pct,pred2_top1_reliability_power_loss_leq_1db_pct,pred3_top1_reliability_power_loss_leq_1db_pct,pred0_top1_reliability_power_loss_leq_3db_pct,pred1_top1_reliability_power_loss_leq_3db_pct,pred2_top1_reliability_power_loss_leq_3db_pct,pred3_top1_reliability_power_loss_leq_3db_pct,pred0_top1_reliability_power_loss_leq_6db_pct,pred1_top1_reliability_power_loss_leq_6db_pct,pred2_top1_reliability_power_loss_leq_6db_pct,pred3_top1_reliability_power_loss_leq_6db_pct,pred0_top1_test_acc_percent,pred0_top3_test_acc_percent,pred0_top5_test_acc_percent,pred1_top1_test_acc_percent,pred1_top3_test_acc_percent,pred1_top5_test_acc_percent,pred2_top1_test_acc_percent,pred2_top3_test_acc_percent,pred2_top5_test_acc_percent,pred3_top1_test_acc_percent,pred3_top3_test_acc_percent,pred3_top5_test_acc_percent
0,8,"['unit2to1_vector', 'unit2_height_log']",23,0.374,0.99,0.362,0.387,0.513,0.897,1.223,1.235,1.248,1.256,84.717,84.556,84.395,84.302,1.317,1.331,1.339,1.339,83.534,83.360,83.267,83.267,1.410,1.428,1.441,1.446,82.381,82.152,81.991,81.929,1.610,1.641,1.661,1.667,79.877,79.487,79.239,79.158,2.203,2.315,2.351,2.377,72.459,71.059,70.612,70.290,95.290,95.538,94.745,93.356,98.314,97.967,97.323,96.529,99.306,99.405,99.058,98.265,86.8,99.7,100.0,87.2,99.7,100.0,85.6,99.5,99.9,84.0,99.3,99.8
1,16,"['unit2to1_vector', 'unit2_height_log']",23,0.502,0.99,0.318,0.306,0.424,0.596,1.357,1.364,1.375,1.388,91.519,91.473,91.406,91.323,1.455,1.457,1.475,1.485,90.909,90.897,90.783,90.719,1.555,1.575,1.595,1.610,90.280,90.154,90.034,89.939,1.785,1.815,1.842,1.865,88.844,88.657,88.485,88.347,2.682,2.836,2.943,3.007,83.238,82.278,81.609,81.204,94.845,95.091,95.140,93.127,98.184,98.184,97.742,97.251,99.509,99.411,98.773,98.969,82.4,98.5,99.5,83.8,98.5,99.2,82.5,98.6,99.5,79.7,98.0,99.2
2,32,"['unit2to1_vector', 'unit2_height_log']",23,1.579,0.99,0.328,0.345,0.397,0.836,1.907,1.897,1.900,1.904,94.040,94.073,94.061,94.049,2.084,2.086,2.095,2.104,93.488,93.482,93.454,93.425,2.393,2.394,2.394,2.396,92.523,92.518,92.520,92.512,2.996,3.007,3.029,3.050,90.637,90.604,90.535,90.468,4.668,4.706,4.774,4.821,85.413,85.293,85.082,84.934,95.268,96.000,95.805,93.415,98.878,99.171,98.244,97.317,99.610,99.463,99.3